In [2]:
import pandas as pd

In [13]:
#path = '/home/lucia/phd_work/shareTask/'
path =  '/Users/lucia/phd_work/Clpsy/'
posts = pd.read_csv(path + 'data/clpsych19_test_data/shared_task_posts_test.csv')
postid = pd.read_csv(path + 'data/clpsych19_test_data/task_B_test.posts.csv')
postid = postid.drop(['user_id', 'subreddit'], axis = 1)

In [27]:
print('posts in Btest' ,postid.shape[0])
print('all the posts in test set', posts.shape[0])

posts in Btest 9610
all the posts in test set 14447


In [67]:
test = pd.merge(postid, posts, on = 'post_id', how = 'left')
test = test.iloc[:,0:6]
print(test.shape)
test.to_csv(path + 'data/clpsych19_training_data/testAll.csv')
test[0:5]

(9610, 6)


,post_id,user_id,timestamp,subreddit,post_title,post_body
0,eer4k,35972,1291242113,DoesAnybodyElse,DAE cross their feet and try to pull them apar...,Let me try to explain before people think it's...
1,exlu5,35972,1294363123,AskReddit,Is their a specific genre of electronic music ...,_PERSON_/watch?v=UsyhCe0V9HMBodyrox ft. Lucian...
2,exmbf,35972,1294364815,electronicmusic,Is their a specific genre of electronic music ...,NaN
3,f64pu,35972,1295567283,AskReddit,What is your purpose in life?,I think my purpose in life is to help my fello...
4,fbyan,35972,1296418935,DoesAnybodyElse,DAE fucking love tomato juice?,NaN


we can see most of the users only have 1 post in SW

In [68]:
testSW = test[test['subreddit'] == 'SuicideWatch']
users = testSW.groupby('user_id')['user_id'].nunique()
testSW.to_csv(path + 'data/clpsych19_training_data/testSW.csv') 
print(testSW.shape)
print('number of users is', users.shape[0])

(186, 6)
number of users is 125


Posts extracted from psyList

In [74]:
def readDictionaries(filePath):
    with open(filePath) as f:
        myList = [x.strip().replace("'","").lower() for x in f.read().split(",")]
        #print(Psylist)
    return myList

def lowerCase(file):
    file['post_title'] = file['post_title'].apply(lambda x: x.lower() if type(x) is str else 'NULL')
    file['subreddit'] = file['subreddit'].apply(lambda x: x.lower() if type(x) is str else 'NULL')
    
    file['post_body'] = file['post_body'].apply(lambda x: x.lower() if type(x) is str else 'NULL')
    
def findText(text, wordList):
    for item in wordList:
        if item in text:
            return True
    return False

def subsetDictPosts(file, dictionary, newTitle, newBody, newSub):
    file[newTitle] = file.apply(lambda row: True if findText(row["post_title"], dictionary) else False, axis=1)
    file[newBody] = file.apply(lambda row: True if findText(row["post_body"], dictionary) else False, axis=1)
    file[newSub] = file.apply(lambda row: True if findText(row["subreddit"], dictionary) else False, axis=1)
    newFile = file[['post_id',newTitle,newBody,newSub]]
    return newFile 
   

lowerCase(test)
PsyList = readDictionaries(path +'suicideDetection/dictionaries/psyList.txt')
file = subsetDictPosts(test, PsyList, 'Psy_title', 'Psy_body', 'Psy_subreddit')
MH = test[(file['Psy_body'] == True) | (file['Psy_title'] == True) | (file['Psy_subreddit'] == True)]
MH.shape
MH[1:5]

,post_id,user_id,timestamp,subreddit,post_title,post_body,Psy_title,Psy_body,Psy_subreddit
38,mr5ui,35972,1322427496,trees,"at a [4], and i have an idea i would really li...",i was talking to a friend just now and we got ...,False,True,False
46,qg7k1,35972,1330803860,adhd,i took my prescribed 15 mg dexedrine spansule ...,friend: how was the midterm?massive_tunes: eas...,False,False,True
50,t0xye,35972,1335841458,favors,[update] motivational speech guy with a very s...,i've responded to all of your messages!! it's ...,False,True,False
51,t2bsu,35972,1335914865,suicidewatch,"as suggested by a fellow redditor, i have reco...","i just finished recording it. ah, i'm actually...",False,False,True


In [81]:
stressed =  MH[MH['subreddit'] != 'suicidewatch']
print(stressed.shape)

(785, 9)


In [82]:
stressed = stressed.iloc[:, 0:6]
print(stressed.shape)
stressed[1:5]


(785, 6)


,post_id,user_id,timestamp,subreddit,post_title,post_body
38,mr5ui,35972,1322427496,trees,"at a [4], and i have an idea i would really li...",i was talking to a friend just now and we got ...
46,qg7k1,35972,1330803860,adhd,i took my prescribed 15 mg dexedrine spansule ...,friend: how was the midterm?massive_tunes: eas...
50,t0xye,35972,1335841458,favors,[update] motivational speech guy with a very s...,i've responded to all of your messages!! it's ...
52,tdkm5,35972,1336512252,getmotivated,if you're feeling down/sad/depressed/unmotivat...,"for context as to why i'm doing this, look at ..."


In [83]:
SWstressed = testSW.append(stressed)
print(SWstressed.shape)
SWstressed.to_csv(path + 'data/clpsych19_training_data/testSWstressed.csv')

(971, 6)


In [115]:
results = pd.read_csv(path + 'data/languageModel/TestResults/test_5gram/rawish_ppl_no_oov.csv')
print(results['predicted_label'].value_counts())
posts = pd.read_csv(path + 'data/clpsych19_training_data/testSW.csv')
results.head(2)

d    88
c    14
b    12
a    11
Name: predicted_label, dtype: int64


,userid,a_score,b_score,c_score,d_score,predicted_label
0,11442,196.124932,192.616578,189.846183,194.692313,c
1,35428,293.207209,179.941954,180.028062,303.571338,b


In [117]:
reAll = pd.merge(results, posts, left_on = 'userid', right_on ='user_id', how = 'right')
reAll.to_csv(path + 'data/clpsych19_training_data/testResults5_rawish_ppl_no_oov.csv')